In [1]:
import pymysql.cursors
import pandas as pd
import uuid

In [4]:
# GET /cluster_analysis
req = json.loads(REQUEST)
args = req['args']

i_process_id = 0
i_process_id = args['process_id'][0]

from datetime import datetime

# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Query parameter of the jobs
sql = "select hastag, parameter from screen_analisis_ai where id = %s"
cursor.execute(sql, i_process_id)

result = cursor.fetchall()
database_keyword_id = result[0]['hastag']
similarity_treshold = result[0]['parameter']
screen_name = ''

#
# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (%s, %s, %s)"
# Execute the query
cursor.execute(sql, (str(i_process_id), datetime.now(), 1 ))

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
cursor.execute(sql, (i_process_id, 1, 'Similarity Treshold', similarity_treshold))
cursor.execute(sql, (i_process_id, 1, 'DB_ID', database_keyword_id))

# Commit Record
connection.commit()

s_query_string = 'select id, tweet, tweet_date_time from ret_tweet where '

if (screen_name != ''):
    # print('use screen name')
    s_query_string = s_query_string + 'screen_name = "' + screen_name + '" and db_id = "' + str(database_keyword_id) + '"'
else:
    # print('no use')
    s_query_string = s_query_string + 'db_id = "' + str(database_keyword_id) + '"'
    
print(s_query_string)
df = pd.read_sql(s_query_string, con=connection)

# Close Connection
connection.close()

# see result
df.head()




import string
import re
import nltk

from nltk.tokenize import word_tokenize

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    
    return strOut
# end Text Preprocessing


# Apply to data frame
df['tweet'] = df['tweet'].apply(text_preproc)

# df.head()



from nltk.corpus import stopwords 

# Load stopwords
sw = stopwords.words('indonesian')

# apply tokenize
df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)
# apply stopword removal
df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

# print((df))


def calculate_similarity(X_set, Y_set):
# Program to measure the similarity between  
# two sentences using cosine similarity. 

    l1 =[];l2 =[]

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    
    return cosine

def largest_in_col(arr,nCol):
    # 
    # Find largest value of col nCol on 2D arr
    #
    
    # init value
    max_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] > max_val:
            max_val = arr[x][nCol]
            row_index = x
        
    return max_val,row_index



import numpy as np
import array as arr
import multitasking
from tqdm import tqdm

# set to max CPU Cores
# multitasking.set_max_threads(multitasking.config["CPU_CORES"] * 5)

st = similarity_treshold
cluster_no = 1
s_score = 0
s_score_current = 0
i_current_cluster = 0

#create zero element array
#col 0 => index base tweet
#col 1 => cluster number
#col 2 => similarity score
base_tweet = []

#proceed to compare to all tweet
cosine_matrix = np.zeros(( len(df), len(df) ))

# flag the mt
with tqdm(total=( (len(df)*len(df)/2))-(len(df)/2))  as pbar:
    for j in range(0, len(df)):
        tweet_to_compare = df['sw'][j]

        #check, is this second tweet?
        if(j == 0):
            #first tweet, add as cluster no #1
            base_tweet.append([j,1,1.0])
            i_current_cluster = base_tweet[0][1]

        elif(j == 1):
            #compare to prev tweet
            s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[0][0] ])

            if(s_score < st):
                #not similar
                base_tweet.append([j,2,1])
                i_current_cluster = base_tweet[j][1]

        else:
            #other else tweet
            for x in range(0,len(base_tweet)):
                #compare every element
                s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[x][0] ])
                base_tweet[x][2] = s_score

            if(largest_in_col(base_tweet,2)[0] < st):
                #no similar, add as one new cluster
                i_current_cluster = i_current_cluster + 1
                base_tweet.append([j,i_current_cluster,largest_in_col(base_tweet,2)[0]])
            else:
                #determine cluster# from biggest similarity
                i_current_cluster = base_tweet[(largest_in_col(base_tweet,2)[1])][1]

        #proceed to compare to all tweet
        for i in range(0,len(df)):
            # update progress
            if (j<i):
                pbar.update(1)
                s_score = calculate_similarity(tweet_to_compare, df['sw'][i])
                if (s_score >= st):
                    cosine_matrix[i,j] = i_current_cluster

        
pbar.close()
# print(cosine_matrix)


# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Base Tweet Record
sql = "insert into ret_base_tweet (job_id, tweet_id, cluster_id) values (%s, %s, %s)"

## inserting base tweet
for i in range(0,len(base_tweet)):
    # Execute the query
    cursor.execute(sql, (i_process_id, df['id'][i], base_tweet[i][1]))
    
connection.commit()
connection.close()

print('finished inserting base tweet record')


# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Tweet Cluster Record
sql = "insert into ret_cluster_result (job_id, tweet_id, cluster_no) values (%s, %s, %s)"

print(len(cosine_matrix))
print(i_process_id)

# initiate cluster number
i_cluster_no_save = 0
temp_val = 0
                 
## inserting tweet cluster
for i in range(0, len(cosine_matrix)):
    
    # find value in this particular row
    for j in range(0,len(cosine_matrix[i])):
        
        # print(cosine_matrix[i][j])
        temp_val = cosine_matrix[i][j]
        
        if(temp_val != 0):
            i_cluster_no_save = temp_val

    # Execute the query
    cursor.execute(sql, (i_process_id, df['id'][i], i_cluster_no_save))
    
connection.commit()
connection.close()

print('finished inserting cluster data')



# Connect to the database
connection = pymysql.connect(host='202.157.176.225',
                             user='cekmedsos_db',
                             password='kuku838485*#',
                             database='cekmedsos_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

#
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
cursor.execute(sql, (i_process_id, 1, '#Tweet Processed',len(df)))

#
# Create Tweet Cluster Record
sql = "update ret_analysis_header set datetime_finish = %s where job_id = %s"
# Executing query
cursor.execute(sql, (datetime.now(),i_process_id) )

print(i_process_id)

connection.commit()
connection.close()

print(json.dumps({'convertedAngle': converted}))

NameError: name 'json' is not defined